In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## New data

In [2]:
label_dict = {4:0, 5:1}

In [3]:
X_train = pd.read_csv("../data/HAPT/Train/X_train.txt", header=None, sep=" ")
y = pd.read_csv("../data/HAPT/Train/y_train.txt", header=None).rename(columns={0:"label"})
X_train["label"] = y.label
X_train = X_train.sample(frac=1.).reset_index(drop=True)

X_test = pd.read_csv("../data/HAPT/Test/X_test.txt", header=None, sep=" ")
y = pd.read_csv("../data/HAPT/Test/y_test.txt", header=None).rename(columns={0:"label"})
X_test["label"] = y.label 

In [4]:
X_train.shape, X_test.shape

((7767, 562), (3162, 562))

In [5]:
np.unique(X_train.label, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([1226, 1073,  987, 1293, 1423, 1413,   47,   23,   75,   60,   90,
          57]))

In [6]:
train_idx = X_train.label.isin([4,5])
np.unique(X_train[train_idx].label, return_counts=True)

(array([4, 5]), array([1293, 1423]))

In [7]:
# Sitting vs standing
train_idx = X_train.label.isin([4,5])
X_train = X_train[train_idx].reset_index(drop=True)
X_train["label"] = X_train["label"].map(label_dict)

test_idx = X_test.label.isin([4,5])
X_test = X_test[test_idx].reset_index(drop=True)
X_test["label"] = X_test["label"].map(label_dict)
y_test = X_test["label"]

In [8]:
X_train.shape, X_test.shape

((2716, 562), (1064, 562))

In [9]:
np.unique(X_train.label, return_counts=True)

(array([0, 1]), array([1293, 1423]))

In [10]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,label
0,0.040243,-0.002684,-0.020638,-0.996505,-0.996423,-0.994309,-0.996900,-0.995687,-0.994611,-0.799694,-0.751905,-0.717516,0.837553,0.715361,0.678068,-0.997548,-0.999980,-0.999981,-0.999947,-0.996691,-0.995407,-0.994355,-0.705828,-0.715289,-0.640649,0.428270,-0.148116,0.278691,0.036127,0.422815,-0.283304,0.348060,0.042574,0.361628,-0.084547,0.170821,-0.038616,0.207362,-0.126003,-0.059313,0.864398,-0.344069,-0.243635,-0.998780,-0.999368,-0.999097,-0.998913,-0.999416,-0.999103,0.794177,-0.367750,-0.250326,0.889534,-0.316061,-0.241643,0.382838,0.648454,-0.787413,-0.877008,-0.999241,-0.999668,-0.999042,-1.000000,-1.0,-1.000000,-0.155429,0.199949,-0.245270,0.291296,0.178666,-0.130571,0.081504,-0.033415,-0.099502,0.140254,-0.180525,0.219565,-0.802173,-0.901742,0.651628,0.076309,0.014078,-0.032759,-0.992358,-0.991581,-0.991173,-0.992196,-0.989468,-0.990399,-0.994181,-0.995586,-0.994846,0.991855,0.993747,0.986682,-0.992835,-0.999919,-0.999884,-0.999831,-0.990833,-0.989467,-0.989358,-0.744950,-0.847730,-0.717519,0.331114,0.008559,0.430480,0.669208,0.395338,-0.183545,0.487499,0.256549,0.230083,0.092803,0.310735,0.364079,0.259515,0.099777,0.037689,-2.696342e-02,-0.019136,-0.079452,-0.994016,-0.995145,-0.996139,-0.994197,-0.995614,-0.996197,-0.909240,-0.956484,-0.771937,0.850448,0.892991,0.826309,-0.994638,-0.999973,-0.999974,-0.999976,-0.995429,-0.996488,-0.995577,-0.484168,-0.469555,-0.689320,0.231368,-0.225231,0.114994,0.142977,0.215015,-0.115741,0.164775,0.247057,0.518008,-0.411013,0.344292,0.062815,0.201294,-0.506591,-0.460108,-0.097991,-0.040614,-0.057046,-0.994474,-0.995201,-0.995705,-0.995326,-0.995594,-0.995629,-0.993477,-0.994591,-0.994698,0.993576,0.996055,0.995755,-0.996374,-0.999964,-0.999977,-0.999965,-0.996388,-0.995471,-0.995834,-0.702549,-0.663793,-0.688431,0.180963,0.028781,0.037117,0.093685,0.204951,0.082543,0.184627,0.485

# Logreg

In [11]:
def get_predictions(x):
    return [1 if xi >= 0.5 else 0 for xi in x]

In [12]:
X, y = X_train[[col for col in X_train.columns if col != "label"]].values , X_train.label

In [13]:
X.shape, y.shape

((2716, 561), (2716,))

In [14]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[train_index, :], X[test_index,:]
    _y_train = y[train_index].values
    clf = LogisticRegression(random_state=0, C=1., max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

/Users/viktortorpthomsen/opt/anaconda3/envs/dont_overfit_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Models AUC score: 0.9926888880555278
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1293
           1       0.97      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [25]:
# Logreg MLE
log_reg = LogisticRegression(random_state=0, C=1.0, max_iter = 500).fit(X[:561,:], y[:561])

In [26]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test)[:,1]

In [27]:
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.9808955984818445


## Small logreg model

In [18]:
sort_idx = np.argsort(np.abs(log_reg.coef_))
sort_coef_vals = np.abs(log_reg.coef_)[0][sort_idx[0]]

In [19]:
len(sort_coef_vals[sort_coef_vals > 0.1])

197

In [20]:
feature_idx = sort_idx[0][-195:]

In [21]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[:,feature_idx][train_index], X[:,feature_idx][test_index]
    _y_train = y[train_index]
    clf = LogisticRegression(random_state=0, C=1., max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.9925252956755631
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1293
           1       0.96      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [22]:
# Logreg MLE
log_reg = LogisticRegression(random_state=0, C=1.0, max_iter = 500).fit(X[:561,feature_idx], y[:561])

In [23]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test[:,feature_idx])[:,1]

In [24]:
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.9794652467002776


# Gaussian logreg

## MAP l2 estimate C=0.3

In [28]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[train_index, :], X[test_index,:]
    _y_train = y[train_index].values
    clf = LogisticRegression(random_state=0, C=.3, max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.9909078507493998
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1293
           1       0.96      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [29]:
log_reg = LogisticRegression(random_state=0, C=.3, penalty="l2", max_iter = 500).fit(X[:561,:], y[:561])

In [30]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test)[:,1]
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.9823011669404633


## Small MAP logreg model

In [31]:
sort_idx = np.argsort(np.abs(log_reg.coef_))
sort_coef_vals = np.abs(log_reg.coef_)[0][sort_idx[0]]

In [20]:
feature_idx = sort_idx[0][-195:]

In [34]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[:,feature_idx][train_index], X[:,feature_idx][test_index]
    _y_train = y[train_index]
    clf = LogisticRegression(random_state=0, C=.3, penalty="l2", max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.990836109240578
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1293
           1       0.96      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [35]:
# Logreg MLE
log_reg = LogisticRegression(random_state=0, C=.3, penalty="l2", max_iter=500).fit(X[:561,feature_idx], y[:561])

In [36]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test[:,feature_idx])[:,1]

In [37]:
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.9799538322098227


In [28]:
X_test.shape

(271, 195)

# Slap-and-spike prior

In [29]:
X, y = X_train[[col for col in X_train.columns if col != "label"]].values , X_train.label

In [30]:
import pymc3 as pm
import theano as tt
from scipy.special import expit
from scipy.stats import norm, bernoulli

This is our slab and spike model

$$a \sim \mathcal{N}(0, 3)$$
$$\gamma_i \sim Bernoulli(p=0.1)$$
$$\alpha_i|\sigma_\beta \sim \mathcal{N}(0, \sigma_\beta)$$
$$e \sim \mathcal{N}(0, \sigma^2_eI_n)$$
$$y \sim \frac{1}{1+exp(-(a + \sum_{i=1}^N \gamma_i \alpha_i x_i + e))}$$

The model parameters are $\theta = \{\gamma_i, \alpha_i\}_i^N $. The $\gamma_i$ and $\alpha_i$ are modelled IID.

In [31]:
prob = 0.1
a_mu = 0
a_var = 3
gamma_var = 1
with pm.Model() as model:
    # priors inclusion probability
    gamma_i = pm.Bernoulli("gamma_i", prob, shape=X.shape[1])
    # a is the interception
    a = pm.Normal("a", mu=a_mu, sd=a_var)
    # The prior for the features varibles which are included
    alpha = pm.Normal("alpha", mu=0, sd=gamma_var, shape=X.shape[1])
    # Deterministic function
    p = pm.math.dot(X,gamma_i * alpha) 
    # Likelihood
    y_obs = pm.Bernoulli("y_obs", pm.invlogit(p + a),  observed=y)
 

In [32]:
with model:
    trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)
    

/var/folders/bx/2b_70g054qg3qt5jhnw30xv40000gn/T/ipykernel_47018/2955332559.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)
Sequential sampling (1 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [gamma_i]
>NUTS: [alpha, a]


Sampling 1 chain for 1_000 tune and 2_000 draw iterations (1_000 + 2_000 draws total) took 2142 seconds.
There were 24 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


# Map estimate

The log loss for the spike and slap prior
$$p(\theta) = p(\{\gamma_i, \alpha_i\}_i^N) = \prod_{i=1}^N p(\gamma_i) p(\alpha_i)$$

$$\log p(\theta) = \sum_{i=1}^N \log Bernoulli(\gamma_i | p=0.1) + \log \mathcal{N}(\alpha_i | \mu=0, \sigma_\beta=3)$$

In [33]:
def spike_slab_log_prior(gamma: np.array, alpha: np.array, p, gamma_mu=0, sigma_beta=3):
    return (bernoulli.logpmf(gamma, p=p) + norm.logpdf(alpha, loc=gamma_mu, scale=sigma_beta)).sum()

The negative log likelihood function, i.e. cross entropy

$$E(\mathbf{w}) = -log(p(\pmb{\mathbf{t}}|\pmb{\mathbf{x}},\mathbf{w})) \nonumber$$

$$= - \sum_{n=1}^N \left( t_n \ln y(\mathbf{x}_n) + (1-t_n) \ln (1-y(\mathbf{x}_n)) \right)$$
  

In [34]:
def log_likelihood(a, gamma, alpha, X, T):
    y_x = expit(a + np.dot(X, np.transpose(gamma*alpha)))
    return (T*np.log(y_x) + ((1-T)*np.log(1-y_x))).sum()

In [35]:
prob = 0.1
gamma_mu = 0
sigma_beta = 3

def find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta):
    min_loss = np.inf
    cur_min = -1
    for i in range(len(trace)):
        tmp_trace = trace[i]
        tmp_spike_slab_log_prior = spike_slab_log_prior(tmp_trace["gamma_i"], tmp_trace["alpha"], p=prob, gamma_mu=gamma_mu, sigma_beta=sigma_beta)
        tmp_log_likelihood = log_likelihood(tmp_trace["a"], tmp_trace["gamma_i"], tmp_trace["alpha"], X, y)
        neq_loss = -(tmp_log_likelihood + tmp_spike_slab_log_prior)
        if neq_loss <= min_loss:
            min_loss = neq_loss
            cur_min = i
    return trace[cur_min]
    

In [36]:
map_trace = find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta)

In [37]:
map_estimate = map_trace["gamma_i"] * map_trace["alpha"]

_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
map_preds = expit(map_trace["a"] + np.dot(_X_test, np.transpose(map_estimate)))

In [47]:
print(f"Models AUC score: {roc_auc_score(y_test, map_preds)}")

Models AUC score: 0.9642659887837761


# Full Bayesian

In [39]:
results = pd.DataFrame({'var': np.arange(561), 
                        'inclusion_probability':np.apply_along_axis(np.mean, 0, trace['gamma_i']),
                       'alpha':np.apply_along_axis(np.mean, 0, trace['alpha']),
                       'alpha_given_inclusion': np.apply_along_axis(np.sum, 0, trace['gamma_i']*trace['alpha'])
                            /np.apply_along_axis(np.sum, 0, trace['gamma_i'])
                       })

In [40]:
results.sort_values('inclusion_probability', ascending = False).head(10)


,var,inclusion_probability,alpha,alpha_given_inclusion
41,41,1.0000,-3.883960,-3.883960
451,451,1.0000,-2.267181,-2.267181
182,182,1.0000,5.423177,5.423177
142,142,1.0000,1.571083,1.571083
50,50,1.0000,-4.035479,-4.035479
445,445,1.0000,4.380210,4.380210
186,186,1.0000,-3.663025,-3.663025
187,187,0.9970,-2.154221,-2.158809
158,158,0.9970,-0.996644,-1.002019
53,53,0.9915,-3.449169,-3.479308


## Bayesian inference

In [41]:
estimate = trace['alpha'] * trace['gamma_i'] 
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
preds = np.apply_along_axis(np.mean, 1, expit(trace['a'] + np.dot(_X_test, np.transpose(estimate) )) )


In [42]:
print(f"Models AUC score: {roc_auc_score(y_test, preds)}")

Models AUC score: 0.9731986064691553


In [43]:
preds[:10]

array([0.99728329, 0.95677686, 0.92627129, 0.96335403, 0.97437116,
       0.94798007, 0.89826115, 0.83813811, 0.84503151, 0.9424179 ])

In [44]:
map_preds[:10]

array([0.99377665, 0.9638406 , 0.86181986, 0.90449025, 0.98654351,
       0.9787043 , 0.94709745, 0.91942937, 0.89949213, 0.92160971])

In [45]:
(map_preds[map_preds > 0.5]).mean(), (map_preds[map_preds <= 0.5]).mean()

(0.9363551762776625, 0.09595969172622476)

In [46]:
(preds[preds > 0.5]).mean(), (preds[preds <= 0.5]).mean()

(0.9245190406644662, 0.09512387902157081)